In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/Users/cfa5228/Documents/Repos/JuliaEpiHandbook"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using DataFrames # We need to load the DataFrames package to create a DataFrame

john_tuple = ("John", 25, 1.8)
john_ntuple = (name = "John", age = 25, height = 1.8)
john_dict = Dict("name" => "John", "age" => 25, "height" => 1.8)
john_df = DataFrame(name = "John", age = 25, height = 1.8);

In [3]:
people_vec = ["John", "Jane", "Joe"]
test_arr = [1 2 3; 4 5 6; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [4]:
john_tuple[1] # "John"
john_ntuple[1] # "John"
people_vec[1]

"John"

In [5]:
test_arr[1, 1]
john_df[1, 1]

"John"

In [6]:
test_arr[:, 1]

3-element Vector{Int64}:
 1
 4
 7

In [7]:
john_df[:, 1]

1-element Vector{String}:
 "John"

In [8]:
test_arr[1, :]

3-element Vector{Int64}:
 1
 2
 3

In [9]:
john_dict["name"]

"John"

In [10]:
john_df[1, :name] # The : operator before the column name turns it into a symbol that can be used to index the dataframe
john_df[1, "name"]

john_ntuple.name

"John"

In [11]:
function multiply_by_two_divide_by_three(x)
    y = x * 2
    z = y / 3
    return z # it's good practice to explicitly return a value (or nothing in special cases)
end

multiply_by_two_divide_by_three (generic function with 1 method)

In [12]:
multiply_by_two_divide_by_three(3)

2.0

In [13]:
multiply_by_two_divide_by_three(10)

6.666666666666667

In [14]:
function multiply_together_offsets(x, y)
    z = (x - 1) * (y + 1)
    return z
end

multiply_together_offsets (generic function with 1 method)

In [15]:
multiply_together_offsets(5, 10)

44

In [16]:
multiply_together_offsets(10, 5)

54

In [17]:
function multiply_together_offsets(x, y; offset_x = 1, offset_y = 1)
    z = (x - offset_x) * (y + offset_y)
    return z
end

multiply_together_offsets (generic function with 1 method)

In [18]:
multiply_together_offsets(5, 10)

44

In [19]:
multiply_together_offsets(5, 10; offset_x = 2, offset_y = 3)

39

In [20]:
function add_one(x)
    y = x + 1
    return y
end

add_one(5)

6

In [21]:
#| error: true
y

LoadError: UndefVarError: `y` not defined

In [22]:
global_x = 5

function print_global_x()
    return println(global_x)
end

print_global_x()

5


In [23]:
function modify_global_x()
    global_x = 10
    return global_x
end

modify_global_x()

global_x

5

In [24]:
function multiply_by_two_divide_by_three(x::Tuple)
    y = zeros(Float64, length(x))
    z = similar(y)

    for i in eachindex(x)
        y[i] = x[i] * 2
        z[i] = y[i] / 3
    end

    return z
end

multiply_by_two_divide_by_three (generic function with 2 methods)

In [25]:
multiply_by_two_divide_by_three((1, 2, 3))

3-element Vector{Float64}:
 0.6666666666666666
 1.3333333333333333
 2.0

In [26]:
multiply_by_two_divide_by_three(3)

2.0

In [27]:
function number_relations(x, y)
    if x < y
        relation = "less than"
    elseif x == y
        relation = "equal to"
    else
        relation = "greater than"
    end
    return println("x is ", relation, " y.")
end

number_relations(2, 1)

x is greater than y.


In [28]:
function number_between(x)
    if x > 0 && x < 10
        println("x is between 0 and 10")
    else
        println("x is not between 0 and 10")
    end
end

number_between(3)

x is between 0 and 10


In [29]:
number_between(11)

x is not between 0 and 10


In [30]:
function number_between2(x)
    if x > 0 && ((x > 10) == false)
        println("x is between 0 and 10")
    else
        println("x is not between 0 and 10")
    end
end

number_between2(3)

x is between 0 and 10


In [31]:
function mycumsum(x)
    y = 0 # Initialize our running total to 0

    # For each number in x, add it to our running total
    for i in x
        y += x[i] # This is equivalent to y = y + x[i]
    end

    return y
end

mycumsum(1:10)

55

In [32]:
function mycumsum2(x)
    y = 0 # Initialize our running total to 0
    i = 1 # Initialize our counter to 1

    # While our running total is less than 100, add the next number to our running total
    while y < 100
        y += x[i] # This is equivalent to y = y + x[i]
        i += 1 # Update our counter so we can add the next number
    end

    return println("We added ", i, " numbers to get to ", y)
end

mycumsum2(1:100)

We added 15 numbers to get to 105
